<a href="https://colab.research.google.com/github/tmarissa/marissa_DATA606/blob/main/ipynb/601_Analysis_State.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA 606 Capstone
## Marissa Tan
### Impact of COVID-19 on the US Housing Market
__Density and Real Estate (State Level)__<br><br>
This notebook:
- Merge the density and real estate dataset.
- Merge Average Listing Price and Average Listing Price Change for 2019 and 2021 and Density. 
    - Real Estate Dataset Cleansed
    - Real Estate Dataset Uncleansed

Note: Rural Percentage is not used because there is no Rural Percentage in the State Level. A state has many areas of different percentages of rural and urban makeup which makes it difficult to classify it to one single category.

In [1]:
import pandas as pd
import matplotlib
from sklearn.cluster import KMeans
import sklearn.cluster as cluster
import sklearn.metrics as metrics
from collections import Counter
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

# Density and Real Estate

# 1. Density

## 1.1 Read CSV Files
Load Density and Real Estate files

In [2]:
# This computes density using the state's land area and its population
Pop_Land = pd.read_csv('State_Pop_Land.csv', index_col=False)
Pop_Land.head()

,FIPS,state,land_area,pop_2020
0,1000,AL,50645.33,5024279
1,2000,AK,570640.95,733391
2,4000,AZ,113594.08,7151502
3,5000,AR,52035.48,3011524
4,6000,CA,155779.22,39538223


## 1.2 Function: Density
The function will calculate the density which is population per land_area  

In [3]:
def density(x, y):
    return int(x/y)

## 1.3 Population Density 2020

In [4]:
Pop_Land['density'] = Pop_Land[['pop_2020', 'land_area']].apply(lambda x: density(*x), axis=1)
Pop_Land.head(5)

,FIPS,state,land_area,pop_2020,density
0,1000,AL,50645.33,5024279,99
1,2000,AK,570640.95,733391,1
2,4000,AZ,113594.08,7151502,62
3,5000,AR,52035.48,3011524,57
4,6000,CA,155779.22,39538223,253


In [5]:
Pop_Land['FIPS'] = Pop_Land['FIPS'].astype('str')

In [6]:
# Make the FIPS uniform into a 5 digit code
Pop_Land['FIPS'] = Pop_Land['FIPS'].apply(lambda x: x.zfill(5))
Pop_Land.sample(3)

,FIPS,state,land_area,pop_2020,density
7,10000,DE,1948.54,989948,508
29,33000,NH,8952.65,1377529,153
17,21000,KY,39486.34,4505836,114


In [7]:
Pop_Land[Pop_Land['state']=='HI']

,FIPS,state,land_area,pop_2020,density
11,15000,HI,6422.63,1455271,226


In [8]:
Pop_Land[Pop_Land['state']=='DC']

,FIPS,state,land_area,pop_2020,density
8,11000,DC,61.05,689545,11294


In [9]:
Pop_Land[Pop_Land['state']=='MA']

,FIPS,state,land_area,pop_2020,density
21,25000,MA,7800.06,7029917,901


In [10]:
Pop_Land[Pop_Land['state']=='CA']

,FIPS,state,land_area,pop_2020,density
4,06000,CA,155779.22,39538223,253


# 2. Real Estate (Cleansed Data)


## 2.1 Average Listing Price, Average Listing Price Change and Density


### 2.1a. For the Year 2019

In [11]:
RE_2019 = pd.read_csv('df_2019.csv', index_col = False)
RE_2019.sample(5)

,FIPS,year,state,county,density,average_listing_price,average_listing_price_mm,average_listing_price_yy,median_listing_price,median_listing_price_mm,median_listing_price_yy,rural_%,rural_cat,total_listing_count
13319,29159,2019,MO,Pettis County,63,188679.0,0.0169,0.0072,147400.0,0.0536,0.1338,37.778726,2,212.0
2038,5071,2019,AR,Johnson County,39,171182.0,0.0016,-0.0914,144000.0,-0.0035,-0.0938,71.366484,1,334.0
9563,20189,2019,KS,Stevens County,7,192252.0,0.2390,0.3547,145000.0,0.1600,0.2083,31.236897,2,26.0
11909,27059,2019,MN,Isanti County,94,271841.0,0.0163,0.0835,249900.0,0.0417,0.1011,62.264650,1,172.0
19753,41059,2019,OR,Umatilla County,24,287250.0,-0.0172,-0.4489,249900.0,-0.0196,0.0522,29.066136,2,245.0


In [12]:
RE_2019[(RE_2019['state']=='AK')].describe()

,FIPS,year,density,average_listing_price,average_listing_price_mm,average_listing_price_yy,median_listing_price,median_listing_price_mm,median_listing_price_yy,rural_%,rural_cat,total_listing_count
count,241.000000,241.0,241.000000,241.000000,237.000000,231.000000,241.000000,237.000000,231.000000,241.000000,241.000000,241.000000
mean,2139.419087,2019.0,7.697095,267353.269710,-0.004567,0.188703,250726.199170,0.002946,0.205995,78.055504,1.174274,132.041494
std,79.783788,0.0,32.163831,64662.052376,0.124289,2.613100,71740.433815,0.184732,2.607087,29.874239,0.380134,265.092386
min,2013.000000,2019.0,0.000000,139080.000000,-0.406300,-0.646500,78970.000000,-0.577800,-0.712100,4.117179,1.000000,0.000000
25%,2070.000000,2019.0,0.000000,220000.000000,-0.029700,-0.153450,212500.000000,-0.020600,-0.111100,59.331919,1.000000,2.000000
50%,2130.000000,2019.0,0.000000,281831.000000,0.000000,-0.015800,252500.000000,0.000000,0.000000,100.000000,1.000000,21.000000
75%,2195.000000,2019.0,2.000000,320734.000000,0.009500,0.098350,309429.000000,0.009200,0.066500,100.000000,1.000000,57.000000
max,2290.000000,2019.0,170.000000,370123.000000,1.021500,39.000000,450000.000000,1.368400,39.000000,100.000000,2.000000,1276.000000


In [13]:
# Average Listing Price Mean for 2019
RE_2019_ALP = RE_2019.groupby(['state', 'density'])['average_listing_price'].mean().reset_index()
RE_2019_ALP.head(5)

,state,density,average_listing_price
0,AK,0,244955.355556
1,AK,1,262505.351351
2,AK,2,329144.000000
3,AK,3,326147.166667
4,AK,4,317021.666667


In [14]:
len(RE_2019_ALP['state'].unique())

51

In [15]:
# Merge with State Density
df_2019_ALP = RE_2019_ALP.merge(Pop_Land, how='inner', left_on='state', 
                                right_on='state').drop(['land_area', 'pop_2020', 'FIPS', 'density_x'], axis=1)
df_2019_ALP = df_2019_ALP.rename(columns ={'density_y':'density'})
df_2019_ALP.head()

,state,average_listing_price,density
0,AK,244955.355556,1
1,AK,262505.351351,1
2,AK,329144.000000,1
3,AK,326147.166667,1
4,AK,317021.666667,1


In [16]:
# Average Listing Price Mean for 2019
RE_2019_ALPY = RE_2019.groupby(['state', 'density'])['average_listing_price_yy'].mean().reset_index()
RE_2019_ALPY.head(5)

,state,density,average_listing_price_yy
0,AK,0,0.379530
1,AK,1,-0.139097
2,AK,2,-0.057052
3,AK,3,0.048867
4,AK,4,0.018992


In [17]:
# Merge with State Density
df_2019_ALPY = RE_2019_ALPY.merge(Pop_Land, how='inner', left_on='state', 
                                right_on='state').drop(['land_area', 'pop_2020', 'FIPS', 'density_x'], axis=1)
df_2019_ALPY = df_2019_ALPY.rename(columns ={'density_y':'density'})
df_2019_ALPY.head()

,state,average_listing_price_yy,density
0,AK,0.379530,1
1,AK,-0.139097,1
2,AK,-0.057052,1
3,AK,0.048867,1
4,AK,0.018992,1


### 2.1b. For the Year 2021

In [18]:
RE_2021 = pd.read_csv('df_2021.csv', index_col = False)
RE_2021.sample(5)

,FIPS,year,state,county,density,average_listing_price,average_listing_price_mm,average_listing_price_yy,median_listing_price,median_listing_price_mm,median_listing_price_yy,rural_%,rural_cat,total_listing_count
540,34013,2021,NJ,Essex County,6843,597369.0,0.0050,0.0893,415000.0,-0.0119,0.0575,0.014541,2,1250.0
20155,42061,2021,PA,Huntingdon County,50,277950.0,0.0611,-0.1820,149900.0,0.0060,-0.1668,68.662470,1,83.0
17905,39019,2021,OH,Carroll County,67,189556.0,-0.1415,-0.1421,159950.0,-0.0586,-0.0300,70.952975,1,64.0
17867,39011,2021,OH,Auglaize County,115,217915.0,-0.0029,0.1771,159900.0,0.0060,0.1587,39.136869,2,114.0
20297,42087,2021,PA,Mifflin County,112,162785.0,-0.0600,0.1383,137450.0,-0.1127,0.2495,50.466989,1,81.0


In [19]:
# Average Listing Price 
RE_2021_ALP = RE_2021.groupby(['state', 'density'])['average_listing_price'].mean().reset_index()
RE_2021_ALP.head(5)

,state,density,average_listing_price
0,AK,0,303301.118644
1,AK,1,369025.032258
2,AK,2,403989.666667
3,AK,3,426217.416667
4,AK,4,424255.666667


In [20]:
# Merge with State Density
df_2021_ALP = RE_2021_ALP.merge(Pop_Land, how='inner', left_on='state', 
                                right_on='state').drop(['land_area', 'pop_2020', 'FIPS', 'density_x'], axis=1)
df_2021_ALP = df_2021_ALP.rename(columns ={'density_y':'density'})
df_2021_ALP.head()

,state,average_listing_price,density
0,AK,303301.118644,1
1,AK,369025.032258,1
2,AK,403989.666667,1
3,AK,426217.416667,1
4,AK,424255.666667,1


In [21]:
# Average Listing Price Change 
RE_2021_ALPY = RE_2021.groupby(['state', 'density'])['average_listing_price_yy'].mean().reset_index()
RE_2021_ALPY.head(5)

,state,density,average_listing_price_yy
0,AK,0,0.349256
1,AK,1,0.452358
2,AK,2,0.296775
3,AK,3,0.130058
4,AK,4,0.223233


In [22]:
# Merge with State Density
df_2021_ALPY = RE_2021_ALPY.merge(Pop_Land, how='inner', left_on='state', 
                                right_on='state').drop(['land_area', 'pop_2020', 'FIPS', 'density_x'], axis=1)
df_2021_ALPY = df_2021_ALPY.rename(columns ={'density_y':'density'})
df_2021_ALPY.head()

,state,average_listing_price_yy,density
0,AK,0.349256,1
1,AK,0.452358,1
2,AK,0.296775,1
3,AK,0.130058,1
4,AK,0.223233,1


## 2.2 Highest Average Listing Price

### 2.2a For the Year **2019**

In [23]:
df_2019_largest = df_2019_ALP.groupby(by=['state', 'density']
                                      )['average_listing_price'].mean().nlargest(10).reset_index()
                                    
df_2019_largest.head(5)

,state,density,average_listing_price
0,HI,226,1.548047e+06
1,DC,11294,9.708098e+05
2,MA,901,7.034988e+05
3,RI,1061,6.959859e+05
4,CO,55,5.272207e+05


### 2.2b For the Year 2021

In [24]:
df_2021_largest = df_2021_ALP.groupby(by=['state', 'density']
                                      )['average_listing_price'].mean().nlargest(10).reset_index()
                                    
df_2021_largest.head(5)

,state,density,average_listing_price
0,HI,226,1.776542e+06
1,RI,1061,9.362832e+05
2,DC,11294,9.340552e+05
3,MA,901,8.659526e+05
4,CO,55,7.246310e+05


### 2.3b Similarities and Differences

In [25]:
similar = (df_2021_largest['state']).isin(df_2019_largest['state'])
df_2021_largest[similar].head(5)

,state,density,average_listing_price
0,HI,226,1.776542e+06
1,RI,1061,9.362832e+05
2,DC,11294,9.340552e+05
3,MA,901,8.659526e+05
4,CO,55,7.246310e+05


In [26]:
# This is the same found in 2019 but not in 2021
df_2019_largest[~(df_2019_largest['state']).isin(df_2019_largest['state'])]

,state,density,average_listing_price


For 2019 and 2021, top 5 highest average listing price consistently are Hawaii (HI), District of Columbia (DC), Massachusetts (MA), Rhode Island (RI), and Colorado (CO). 
Pre-COVID, the HI has the highest average listing price 1.5M. It is followed by DC (970K), MA (703K),  RI (695K), and CO (527K). Post_COVID, HI still has the highest average listing price 1.77M. DC has a very high rural percentage was displaced from 2nd to 3rd from 2019 to 2021 by Rhode Island (936K). Because its average listing price was lower to 935K while other states increased their average lisiting price by at least by 100K.


## 2.3 Highest Average Listing Price Change

### 2.3a For the Year **2019**

In [27]:
df_2019_largestY = df_2019_ALPY.groupby(by=['state', 'density']
                                      )['average_listing_price_yy'].mean().nlargest(10).reset_index()
                                    
df_2019_largestY.head(5)

,state,density,average_listing_price_yy
0,NE,25,0.162207
1,ID,22,0.121948
2,GA,186,0.109083
3,OK,57,0.100725
4,WA,115,0.089485


### 2.2b For the Year 2021

In [28]:
df_2021_largestY = df_2021_ALPY.groupby(by=['state', 'density']
                                      )['average_listing_price_yy'].mean().nlargest(10).reset_index()
                                    
df_2021_largestY.head(5)

,state,density,average_listing_price_yy
0,ID,22,0.573245
1,MT,7,0.381044
2,UT,39,0.358258
3,ND,11,0.342563
4,AZ,62,0.311393


### 2.3b Similarities and Differences

In [29]:
similar = (df_2021_largestY['state']).isin(df_2019_largestY['state'])
df_2021_largestY[similar].head(5)

,state,density,average_listing_price_yy
0,ID,22,0.573245


In [30]:
# This is the same found in 2019 but not in 2021
df_2019_largestY[~(df_2019_largestY['state']).isin(df_2019_largestY['state'])]

,state,density,average_listing_price_yy


In 2019, the top 5 states with the highest listing price change begins with Nebraska, Idaho, Georgia, Oklahoma, and Washington. The highest change is below 17% and states that has the highest is less than 200. In 2021, the average listing price change is below 60%. All the states, Idaho, Montana, Utah, North Dakota, and Arizona, in the top 5 highest listing price change has a density of less than 70. 

# 3. Real Estate (Uncleansed Data)

## 3.1 Average Listing Price, Average List Price Change, and Density


### 3.1a. For the Year 2019

In [32]:
RE2_2019 = pd.read_csv('df_outliers_2019.csv', index_col = False)
RE2_2019.sample(5)

,FIPS,year,state,county,density,average_listing_price,average_listing_price_mm,average_listing_price_yy,median_listing_price,median_listing_price_mm,median_listing_price_yy,rural_%,rural_cat,total_listing_count
31191,48225,2019,TX,Houston County,17,194505.0,-0.0061,-0.0812,149000.0,0.0000,-0.2032,73.487275,1,129.0
915,1105,2019,AL,Perry County,11,308377.0,0.0596,0.9773,131250.0,-0.0094,0.0104,100.000000,1,17.0
8404,17029,2019,IL,Coles County,92,123484.0,-0.0160,0.0159,94000.0,0.0301,-0.0105,24.288605,2,266.0
26637,42001,2019,PA,Adams County,200,275000.0,0.0166,-0.0364,239900.0,0.0455,0.0218,53.675782,1,325.0
16282,27105,2019,MN,Nobles County,31,157856.0,-0.0722,-0.0489,139900.0,-0.1948,-0.1251,40.986060,2,80.0


In [33]:
RE2_2019[(RE2_2019['state']=='AK')].describe()

,FIPS,year,density,average_listing_price,average_listing_price_mm,average_listing_price_yy,median_listing_price,median_listing_price_mm,median_listing_price_yy,rural_%,rural_cat,total_listing_count
count,291.000000,291.0,291.000000,291.000000,284.000000,272.000000,291.000000,284.000000,272.000000,291.000000,291.000000,291.000000
mean,2139.175258,2019.0,8.690722,302432.323024,-0.006942,0.192340,285778.862543,0.001736,0.221134,72.315829,1.247423,127.536082
std,77.415851,0.0,33.671648,138780.461344,0.123720,2.435368,148878.000831,0.169315,2.445518,32.917285,0.432258,263.423773
min,2013.000000,2019.0,0.000000,121500.000000,-0.423100,-0.646500,78970.000000,-0.577800,-0.712100,4.117179,1.000000,0.000000
25%,2070.000000,2019.0,0.000000,221751.000000,-0.032025,-0.151975,214150.000000,-0.020650,-0.111100,50.293837,1.000000,2.000000
50%,2130.000000,2019.0,0.000000,296838.000000,-0.000150,-0.000200,264357.000000,0.000000,0.000000,100.000000,1.000000,23.000000
75%,2195.000000,2019.0,2.000000,330900.000000,0.011800,0.095075,330000.000000,0.009400,0.072525,100.000000,1.000000,62.000000
max,2290.000000,2019.0,170.000000,975000.000000,1.021500,39.000000,975000.000000,1.368400,39.000000,100.000000,2.000000,1276.000000


In [34]:
# Average Listing Price Mean for 2019
RE2_2019_ALP = RE2_2019.groupby(['state', 'density'])['average_listing_price'].mean().reset_index()
RE2_2019_ALP.head(5)

,state,density,average_listing_price
0,AK,0,276819.326797
1,AK,1,255966.615385
2,AK,2,408342.358974
3,AK,3,326147.166667
4,AK,4,317021.666667


In [35]:
len(RE2_2019_ALP['state'].unique())

51

In [36]:
# Merge with State Density
df2_2019_ALP = RE2_2019_ALP.merge(Pop_Land, how='inner', left_on='state', 
                                right_on='state').drop(['land_area', 'pop_2020', 'FIPS', 'density_x'], axis=1)
df2_2019_ALP = df2_2019_ALP.rename(columns ={'density_y':'density'})
df2_2019_ALP.head()

,state,average_listing_price,density
0,AK,276819.326797,1
1,AK,255966.615385,1
2,AK,408342.358974,1
3,AK,326147.166667,1
4,AK,317021.666667,1


In [37]:
# Average Listing Price Mean for 2019
RE2_2019_ALPY = RE2_2019.groupby(['state', 'density'])['average_listing_price_yy'].mean().reset_index()
RE2_2019_ALPY.head(5)

,state,density,average_listing_price_yy
0,AK,0,0.413686
1,AK,1,-0.139097
2,AK,2,-0.050631
3,AK,3,0.048867
4,AK,4,0.018992


In [38]:
# Merge with State Density
df2_2019_ALPY = RE2_2019_ALPY.merge(Pop_Land, how='inner', left_on='state', 
                                right_on='state').drop(['land_area', 'pop_2020', 'FIPS', 'density_x'], axis=1)
df2_2019_ALPY = df2_2019_ALPY.rename(columns ={'density_y':'density'})
df2_2019_ALPY.head()

,state,average_listing_price_yy,density
0,AK,0.413686,1
1,AK,-0.139097,1
2,AK,-0.050631,1
3,AK,0.048867,1
4,AK,0.018992,1


### 3.1b For the Year 2021

In [39]:
RE2_2021 = pd.read_csv('df_outliers_2021.csv', index_col = False)
RE2_2021.sample(5)

,FIPS,year,state,county,density,average_listing_price,average_listing_price_mm,average_listing_price_yy,median_listing_price,median_listing_price_mm,median_listing_price_yy,rural_%,rural_cat,total_listing_count
8749,17093,2021,IL,Kendall County,411,426797.0,0.0156,0.3656,337998.0,0.0543,0.2074,10.419572,2,563.0
23826,36063,2021,NY,Niagara County,407,287639.0,0.1430,0.3074,182900.0,-0.0114,0.0457,22.440627,2,856.0
36180,54003,2021,WV,Berkeley County,380,330921.0,-0.0016,0.1122,287450.0,0.0086,0.1060,31.625532,2,672.0
31158,48229,2021,TX,Hudspeth County,0,99000.0,0.0000,-0.4787,99000.0,0.0000,-0.4787,100.000000,1,1.0
3818,12011,2021,FL,Broward County,1607,850400.0,0.0574,0.3011,387000.0,0.0184,0.2113,0.016933,2,11287.0


In [40]:
# Average Listing Price 
RE2_2021_ALP = RE2_2021.groupby(['state', 'density'])['average_listing_price'].mean().reset_index()
RE2_2021_ALP.head(5)

,state,density,average_listing_price
0,AK,0,312031.960938
1,AK,1,384757.000000
2,AK,2,734775.722222
3,AK,3,426217.416667
4,AK,4,424255.666667


In [41]:
# Merge with State Density
df2_2021_ALP = RE2_2021_ALP.merge(Pop_Land, how='inner', left_on='state', 
                                right_on='state').drop(['land_area', 'pop_2020', 'FIPS', 'density_x'], axis=1)
df2_2021_ALP = df2_2021_ALP.rename(columns ={'density_y':'density'})
df2_2021_ALP.head()

,state,average_listing_price,density
0,AK,312031.960938,1
1,AK,384757.000000,1
2,AK,734775.722222,1
3,AK,426217.416667,1
4,AK,424255.666667,1


In [42]:
# Average Listing Price Change 
RE2_2021_ALPY = RE2_2021.groupby(['state', 'density'])['average_listing_price_yy'].mean().reset_index()
RE2_2021_ALPY.head(5)

,state,density,average_listing_price_yy
0,AK,0,0.374722
1,AK,1,0.451000
2,AK,2,0.766997
3,AK,3,0.130058
4,AK,4,0.223233


In [43]:
# Merge with State Density
df2_2021_ALPY = RE2_2021_ALPY.merge(Pop_Land, how='inner', left_on='state', 
                                right_on='state').drop(['land_area', 'pop_2020', 'FIPS', 'density_x'], axis=1)
df2_2021_ALPY = df2_2021_ALPY.rename(columns ={'density_y':'density'})
df2_2021_ALPY.head()

,state,average_listing_price_yy,density
0,AK,0.374722,1
1,AK,0.451000,1
2,AK,0.766997,1
3,AK,0.130058,1
4,AK,0.223233,1


## 3.2 Highest Average Listing Price

### 3.2a For the Year **2019**

In [44]:
df2_2019_largest = df2_2019_ALP.groupby(by=['state', 'density']
                                      )['average_listing_price'].mean().nlargest(10).reset_index()
                                    
df2_2019_largest.head(5)

,state,density,average_listing_price
0,HI,226,1.548047e+06
1,MA,901,1.066211e+06
2,DC,11294,9.708098e+05
3,CA,253,8.960449e+05
4,CO,55,7.563096e+05


### 3.2b For the Year 2021

In [45]:
df2_2021_largest = df2_2021_ALP.groupby(by=['state', 'density']
                                      )['average_listing_price'].mean().nlargest(10).reset_index()
                                    
df2_2021_largest.head(5)

,state,density,average_listing_price
0,HI,226,1.776542e+06
1,MA,901,1.361563e+06
2,CA,253,1.199067e+06
3,CO,55,1.031336e+06
4,RI,1061,9.362832e+05


### 3.3b Similarities and Differences

In [46]:
similar = (df2_2021_largest['state']).isin(df2_2019_largest['state'])
df2_2021_largest[similar].head(5)

,state,density,average_listing_price
0,HI,226,1.776542e+06
1,MA,901,1.361563e+06
2,CA,253,1.199067e+06
3,CO,55,1.031336e+06
4,RI,1061,9.362832e+05


In [47]:
# This is the same found in 2019 but not in 2021
df2_2019_largest[~(df2_2019_largest['state']).isin(df2_2019_largest['state'])]

,state,density,average_listing_price


## 3.3 Highest Average Listing Price Change

### 3.3a For the Year 2019

In [48]:
df2_2019_largestY = df2_2019_ALPY.groupby(by=['state', 'density']
                                      )['average_listing_price_yy'].mean().nlargest(10).reset_index()
                                    
df2_2019_largestY.head(5)

,state,density,average_listing_price_yy
0,ND,11,0.344414
1,NE,25,0.171482
2,OK,57,0.141890
3,ID,22,0.136018
4,NV,28,0.127035


### 3.2b For the Year 2021

In [49]:
df2_2021_largestY = df2_2021_ALPY.groupby(by=['state', 'density']
                                      )['average_listing_price_yy'].mean().nlargest(10).reset_index()
                                    
df2_2021_largestY.head(5)

,state,density,average_listing_price_yy
0,ID,22,0.553776
1,NE,25,0.549333
2,MT,7,0.443744
3,SD,11,0.399667
4,UT,39,0.372320


### 3.3b Similarities and Differences

In [50]:
similar = (df2_2021_largestY['state']).isin(df2_2019_largestY['state'])
df2_2021_largestY[similar].head(5)

,state,density,average_listing_price_yy
0,ID,22,0.553776
1,NE,25,0.549333
6,GA,186,0.331354
8,NV,28,0.300215


In [51]:
# This is the same found in 2019 but not in 2021
df2_2019_largestY[~(df2_2019_largestY['state']).isin(df2_2019_largestY['state'])]

,state,density,average_listing_price_yy


# 4. Save to CSV
- Average Listing Price (cleansed) for 2019 and 2021
  - df_2019_ALP.csv and df_2021_ALP.csv
- Average Listing Price (uncleansed with outliers) for 2019 and 2021
  - df_outliers_2019_ALP.csv and df_outliers_2021_ALP.csv

In [52]:
df_2019_ALP.to_csv('df_2019_ALP.csv', index=False)
df_2021_ALP.to_csv('df_2021_ALP.csv', index=False)
df2_2019_ALP.to_csv('df_outliers_2019_ALP.csv', index=False)
df2_2021_ALP.to_csv('df_outliers_2021_ALP.csv', index=False)